In [8]:
import numpy as np
import os
import pandas as pd
import math
from random import shuffle
from sklearn import datasets

In [9]:
class ItemCF(object):
    #def __init__(self, data_file_path):
     #   self.data_file_path = data_file_path
        
    dataset = dict()
    movie_similarity = {}
        
        
    def __init__(self, data_file_path):
        rating_column_name = ['user_id', 'movie_id', 'rating', 'timestamp']
        ratings = pd.read_table(data_file_path, sep = '::', header = None, names = rating_column_name, engine = 'python')
        #dataset = dict()
        

        for row in ratings.itertuples():
            user_id = getattr(row, 'user_id')
            movie_id = getattr(row, 'movie_id')
            rate = getattr(row, 'rating')
            self.dataset.setdefault(user_id, {})
            self.dataset[user_id][movie_id] = rate
            
        print('complete')
    
    
    #def __item_similarity(self):
        #評分次數
        movie_rate_count = dict()

        #共現矩陣
        movie_cuv_matrix = dict()

        for user_id, movie_ids in self.dataset.items():
            for i in movie_ids.keys():
                movie_rate_count.setdefault(i, 0)
                movie_rate_count[i] += 1

                for j in movie_ids.keys():
                    if i == j:
                        continue

                    movie_cuv_matrix.setdefault(i, dict())
                    movie_cuv_matrix[i].setdefault(j, 0)
                    movie_cuv_matrix[i][j] += 1
        print('complete')

        for i, movie_ids in movie_cuv_matrix.items():
            self.movie_similarity.setdefault(i, dict())

            for j, cuv in movie_ids.items():
                similarity = cuv / math.sqrt(movie_rate_count[i] * movie_rate_count[j])
                self.movie_similarity[i].setdefault(j, 0)
                self.movie_similarity[i][j] = similarity
                #print(similarity)
        print('complete')
                
    def recommend(self, user_id):
        
        rank = dict()
        top_n_movies = dict()
        user_rate_movies = self.dataset.get(user_id, {})
        
        for i, rate in user_rate_movies.items():
            #print(self.movie_similarity)
            top_n_movies = dict(sorted(self.movie_similarity[i].items(),
            key = lambda x:x[1], reverse = True)[0:10])

            for j, similarity in top_n_movies.items():
                if j in user_rate_movies:
                    continue

                rank.setdefault(j, 0)
                rank[j] += rate * similarity

        result = dict(sorted(rank.items(), key = lambda x:x[1], reverse = True)[0:5])
        return result

In [10]:
if __name__ == "__main__":
    itemCF = ItemCF(('ml-1m/ratings.dat'))
    print('start')
    result = itemCF.recommend(1)
    
    movie_column_name = ['movie_id','title', 'movie_type']
    print(movie_column_name)
    movies = pd.read_table('ml-1m/movies.dat', sep = '::', header = None, names = movie_column_name, engine = 'python')
    
    movie_data = dict()
    for row in movies.itertuples():
        movie_id = getattr(row, 'movie_id')
        title = getattr(row, 'title')
        movie_data[movie_id] = title
        
    for movie_id, rate in result.items():
        print('Movie ID:', movie_id, " / Title:", movie_data[movie_id], " / Rate:", rate)

complete
complete
complete
start
['movie_id', 'title', 'movie_type']
Movie ID: 1196  / Title: Star Wars: Episode V - The Empire Strikes Back (1980)  / Rate: 24.670842979575504
Movie ID: 364  / Title: Lion King, The (1994)  / Rate: 20.635769994037044
Movie ID: 318  / Title: Shawshank Redemption, The (1994)  / Rate: 19.113343371149703
Movie ID: 296  / Title: Pulp Fiction (1994)  / Rate: 19.072718148521417
Movie ID: 2080  / Title: Lady and the Tramp (1955)  / Rate: 18.982200112615864
